In [1]:
import os
import sys
os.environ['SPARK_HOME'] = '/Users/seonghwi/spark-2.4.8-bin-hadoop2.7'
print(sys.version)
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession


3.7.9 (v3.7.9:13c94747c7, Aug 15 2020, 01:31:08) 
[Clang 6.0 (clang-600.0.57)]


In [2]:
spark = SparkSession.builder \
    .appName("CSV Loader") \
    .getOrCreate()


24/05/26 16:53:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.csv("/Users/seonghwi/workspace/iphone14_soldout_Post_site.csv", header=True, inferSchema=True)


In [4]:
df.show()
df.printSchema()


+---------------------------------+----------------------------------+--------------------+--------------------+--------+------+------+----+
|                            title|                           context|               price|         upload_date|location|status|imgUrl| url|
+---------------------------------+----------------------------------+--------------------+--------------------+--------+------+------+----+
|            아이폰14 퍼플 128기가|        128기가 배터리90프로입니다|                null|                null|    null|  null|  null|null|
|    찍힌곳 없이 깨끗한 상태입니다|                              null|                null|                null|    null|  null|  null|null|
|                  ,600000,4일 전,|                              null|                null|                null|    null|  null|  null|null|
|                        정발산동"|                           soldout|https://img2.joon...|https://web.joong...|    null|  null|  null|null|
|         아이폰 14 128기가 자급제|           *실사용기간 : 1년

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType,BooleanType
def check_crack(context):
    crack_keywords = ['깨짐', '금', '크랙', '파손']  
    for keyword in crack_keywords:
        if keyword in title:
            return True
    return False 


categorize_udf = udf(categorize_title, StringType())
crack_udf = udf(check_crack, BooleanType())


df = df.withColumn("category", categorize_udf(df["title"]))
df = df.withColumn("has_crack", crack_udf(df["title"]))


df.show()

+---------------------------------+----------------------------------+--------------------+--------------------+--------+------+------+----+--------+---------+
|                            title|                           context|               price|         upload_date|location|status|imgUrl| url|category|has_crack|
+---------------------------------+----------------------------------+--------------------+--------------------+--------+------+------+----+--------+---------+
|            아이폰14 퍼플 128기가|        128기가 배터리90프로입니다|                null|                null|    null|  null|  null|null|    기타|    false|
|    찍힌곳 없이 깨끗한 상태입니다|                              null|                null|                null|    null|  null|  null|null|    기타|    false|
|                  ,600000,4일 전,|                              null|                null|                null|    null|  null|  null|null|    기타|    false|
|                        정발산동"|                           soldout|https://img2

In [13]:
output_path = "/Users/seonghwi/workspace/iphone14_soldout_Post_site_categorized.csv"
df.write.csv(output_path, header=True, mode='overwrite')

